In [1]:
# 数据处理工具库
import numpy as np
import pandas as pd

In [9]:
# 查看feed数据
feed_df = pd.read_csv("../wechat_algo_data/feed_info.csv")
print("feed数据量：{}".format(feed_df.shape))
feed_df.head(2)

feed数据量：(106444, 15)


,feedid,authorid,videoplayseconds,description,ocr,asr,bgm_song_id,bgm_singer_id,manual_keyword_list,machine_keyword_list,manual_tag_list,machine_tag_list,description_char,ocr_char,asr_char
0,43549,6165,38,104741 122649 8109 117252 65632 23463 118668 4...,139499 59421 82007 142955 27736 83577 52394 11...,142955 27736 83577 103956 32010 34170 89740 90...,19356.0,11703.0,15506;7715;17582,26334;219;25209;7715;18541,81;269;159;6,269 0.8525666;81 0.8525666;8 1.1e-07;306 0.0;2...,26439 5247 6426 3827 1882 26018 20744 22204 30...,25926 8491 13394 2203 26439 6243 33054 16435 1...,2203 26439 6243 33054 16435 16307 17070 24908 ...
1,77432,9386,60,35753 27736 146603 73055 11794 101761 11794 81...,35753 146603 73055 11794 101761 67496 16933 52...,146739 14368 79290 79213 47366 8109 33194 1198...,NaN,NaN,8199;18322;4243,24078;19924,194;267;159;6,267 0.99293476;194 0.99293476,31010 32495 6243 13923 15360 30483 2709 26084 ...,31010 32495 13923 15360 30483 2709 26084 15160...,7259 20851 5061 26207 17573 17531 15117 20072 ...


In [10]:
# action数据
action_df = pd.read_csv('../wechat_algo_data/user_action.csv')
print("action数据量：{}".format(action_df.shape))
action_df.head(2)

action数据量：(7317882, 13)


,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite
0,8,71474,1,1,0,0,1,500,5366,0,0,0,0
1,8,73916,1,1,0,0,0,250,1533,0,0,0,0


In [17]:
# 预测数据
test = pd.read_csv('../wechat_algo_data/test_a.csv')
print("预测数据数据量：{}".format(test.shape))
test.head(2)

预测数据数据量：(421985, 3)


,userid,feedid,device
0,14298,67227,1
1,68356,91864,2


In [12]:
import os
import copy
# import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
# 对list字段进行切分和映射编码
def split(column):
    if not isinstance(column,str):
        return []
    keys = column.strip().split(';')
    for key in keys:
        if key not in key2index:
            key2index[key] = len(key2index) + 1
    return list(map(lambda x: key2index[x], keys))

In [14]:
def preprocess(sample,dense_features):
    '''
    特征工程：对数值型特征做对数变换; id型特征+1; 缺失值补充0。
    '''
    sample[dense_features] = sample[dense_features].fillna(0.0)
    sample[dense_features] = np.log(sample[dense_features] + 1.0)
    
    sample[["authorid", "bgm_song_id", "bgm_singer_id"]] += 1  # 0 用于填未知
    sample[["authorid", "bgm_song_id", "bgm_singer_id", "videoplayseconds"]] = sample[["authorid", "bgm_song_id", "bgm_singer_id", "videoplayseconds"]].fillna(0)
    sample["videoplayseconds"] = np.log(sample["videoplayseconds"] + 1.0)
    sample[["authorid", "bgm_song_id", "bgm_singer_id"]] = sample[["authorid", "bgm_song_id", "bgm_singer_id"]].astype(int)
    return sample

In [15]:
# 合并数据
test['date_'] = 15
action_df = pd.concat([action_df,test])

In [18]:
# 标签列
target = ["read_comment", "like", "click_avatar", "forward"]
# 稀疏特征
sparse_features = ['userid', 'feedid', 'authorid', 'bgm_song_id', 'bgm_singer_id']
# 变长序列特征
varlen_features = ['manual_tag_list','manual_keyword_list']
# 稠密特征
dense_features = ['videoplayseconds']

In [20]:
# 数据合并
feed_df = feed_df[['feedid', 'authorid', 'videoplayseconds', 'bgm_song_id', 'bgm_singer_id','manual_tag_list','manual_keyword_list']]
data = action_df.merge(feed_df, how='left',on='feedid') #行为数据拼接，作者id，bgm_song_id 
data = preprocess(data,dense_features) #特征处理
data = data[dense_features+sparse_features+varlen_features+['date_']+target]
# 变长特征编码
encoder = {}
global key2index
for f in ['manual_keyword_list','manual_tag_list']:
    key2index = {}
    f_list = list(map(split, data[f].values))
    f_length = np.array(list(map(len, f_list)))
    max_len = max(f_length)
    print(f'{f} 字段最长的取值序列长度为 {max_len}')
    # Notice : padding=`post`
    data[f] = list(pad_sequences(f_list, maxlen=max_len, padding='post', ))
    encoder[f] = copy.copy(key2index)

manual_keyword_list 字段最长的取值序列长度为 18
manual_tag_list 字段最长的取值序列长度为 11


In [21]:
# 稀疏特征编码
for featid in sparse_features:
    print(f"编码ID字段：{featid}")
    encoder[featid] = {uid:ucode+1 for ucode,uid in enumerate(data[featid].unique())} 
    data[featid] = data[featid].apply(lambda x: encoder[featid].get(x,0))

编码ID字段：userid
编码ID字段：feedid
编码ID字段：authorid
编码ID字段：bgm_song_id
编码ID字段：bgm_singer_id


In [22]:
print('数据维度：', data.shape)
print('数据字段：', data.columns.tolist())
print('不同的date_取值: ', data['date_'].unique())
# 如果资源比较少，可以在这里进行数据采样
data = data.sample(frac = 1.0)

数据维度： (7739867, 13)
数据字段： ['videoplayseconds', 'userid', 'feedid', 'authorid', 'bgm_song_id', 'bgm_singer_id', 'manual_tag_list', 'manual_keyword_list', 'date_', 'read_comment', 'like', 'click_avatar', 'forward']
不同的date_取值:  [ 1  2  3  5  6  7  8 10 11 12 13 14  4  9 15]


In [23]:
# 或者手动创建文件夹data_and_feature
!mkdir data_and_feature

In [24]:
# 构建训练集，验证集和测试集
# 第14天样本作为验证集
train = data[data['date_'] < 14].drop(['date_'],axis = 1)
val = data[data['date_'] == 14].drop(['date_'],axis = 1)  
test = data[data['date_'] == 15].drop(['date_'],axis = 1)

In [25]:
import gc
import joblib
del action_df
del feed_df
del data
gc.collect()

0

In [26]:
joblib.dump(train, './data_and_feature/train.txt')
joblib.dump(val, './data_and_feature/val.txt')
joblib.dump(test, './data_and_feature/test.txt')
joblib.dump(encoder, './data_and_feature/encoder.txt')

['./data_and_feature/encoder.txt']